### Import all python Liberies

In [1]:
import pandas as pd
import numpy as np
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

### Create spark session and load json files spark Dataframe

In [2]:
spark = SparkSession.Builder().appName("Loan credit card").getOrCreate()

df= spark.read.json("datasets/cdw_sapp_branch.json")
df.show(5)

+-----------------+-----------+------------+------------+------------+-----------------+----------+--------------------+
|      BRANCH_CITY|BRANCH_CODE| BRANCH_NAME|BRANCH_PHONE|BRANCH_STATE|    BRANCH_STREET|BRANCH_ZIP|        LAST_UPDATED|
+-----------------+-----------+------------+------------+------------+-----------------+----------+--------------------+
|        Lakeville|          1|Example Bank|  1234565276|          MN|     Bridle Court|     55044|2018-04-18T16:51:...|
|          Huntley|          2|Example Bank|  1234618993|          IL|Washington Street|     60142|2018-04-18T16:51:...|
|SouthRichmondHill|          3|Example Bank|  1234985926|          NY|    Warren Street|     11419|2018-04-18T16:51:...|
|       Middleburg|          4|Example Bank|  1234663064|          FL| Cleveland Street|     32068|2018-04-18T16:51:...|
|    KingOfPrussia|          5|Example Bank|  1234849701|          PA|      14th Street|     19406|2018-04-18T16:51:...|
+-----------------+-----------+-

In [3]:
spark

### Print details about the dataFrame

In [4]:
print(f"rows: {df.count()}")
print(len(df.columns))
df.describe().show()
df.printSchema()
df.columns

rows: 115
8
+-------+-----------+-----------------+------------+--------------------+------------+-------------+------------------+--------------------+
|summary|BRANCH_CITY|      BRANCH_CODE| BRANCH_NAME|        BRANCH_PHONE|BRANCH_STATE|BRANCH_STREET|        BRANCH_ZIP|        LAST_UPDATED|
+-------+-----------+-----------------+------------+--------------------+------------+-------------+------------------+--------------------+
|  count|        115|              115|         115|                 115|         115|          115|               115|                 115|
|   mean|       null|76.67826086956522|        null|1.2345499259478261E9|        null|         null|  38975.2347826087|                null|
| stddev|       null|52.94113709535237|        null|  258751.74757815443|        null|         null|23938.156819564818|                null|
|    min|    Acworth|                1|Example Bank|          1234105725|          AL|  11th Street|              2155|2018-04-18T16:51:...|
|

['BRANCH_CITY',
 'BRANCH_CODE',
 'BRANCH_NAME',
 'BRANCH_PHONE',
 'BRANCH_STATE',
 'BRANCH_STREET',
 'BRANCH_ZIP',
 'LAST_UPDATED']

### If the source value is null load default (99999) value else Direct move

In [5]:
df = df.withColumn("BRANCH_ZIP", when(col("BRANCH_ZIP").isNull(), lit("99999")).otherwise(col("BRANCH_ZIP")))

In [6]:
df = df.withColumn("BRANCH_ZIP", col("BRANCH_ZIP").cast(IntegerType()))
df = df.withColumn("BRANCH_CODE", col("BRANCH_CODE").cast(IntegerType()))
df = df.withColumn("LAST_UPDATED", col("LAST_UPDATED").cast(TimestampType()))

df.dtypes

[('BRANCH_CITY', 'string'),
 ('BRANCH_CODE', 'int'),
 ('BRANCH_NAME', 'string'),
 ('BRANCH_PHONE', 'string'),
 ('BRANCH_STATE', 'string'),
 ('BRANCH_STREET', 'string'),
 ('BRANCH_ZIP', 'int'),
 ('LAST_UPDATED', 'timestamp')]

### Change the format of phone number to (XXX)XXX-XXXX

In [7]:
df = df.withColumn("BRANCH_PHONE",
                                     concat(lit("("),
                                            substring(df.BRANCH_PHONE, 1, 3),
                                            lit(") "),
                                            substring(df.BRANCH_PHONE, 4, 3),
                                            lit("-"),
                                            substring(df.BRANCH_PHONE, 7, 4)))
df.show(truncate=False)

+-----------------+-----------+------------+--------------+------------+-------------------+----------+-------------------+
|BRANCH_CITY      |BRANCH_CODE|BRANCH_NAME |BRANCH_PHONE  |BRANCH_STATE|BRANCH_STREET      |BRANCH_ZIP|LAST_UPDATED       |
+-----------------+-----------+------------+--------------+------------+-------------------+----------+-------------------+
|Lakeville        |1          |Example Bank|(123) 456-5276|MN          |Bridle Court       |55044     |2018-04-18 16:51:47|
|Huntley          |2          |Example Bank|(123) 461-8993|IL          |Washington Street  |60142     |2018-04-18 16:51:47|
|SouthRichmondHill|3          |Example Bank|(123) 498-5926|NY          |Warren Street      |11419     |2018-04-18 16:51:47|
|Middleburg       |4          |Example Bank|(123) 466-3064|FL          |Cleveland Street   |32068     |2018-04-18 16:51:47|
|KingOfPrussia    |5          |Example Bank|(123) 484-9701|PA          |14th Street        |19406     |2018-04-18 16:51:47|
|Paterso

### Rearranging the columns

In [8]:
CDW_SAPP_BRANCH = df.select("BRANCH_CODE","BRANCH_NAME","BRANCH_STREET","BRANCH_CITY","BRANCH_STATE","BRANCH_ZIP","BRANCH_PHONE","LAST_UPDATED")

### Displaying the DF

In [9]:
CDW_SAPP_BRANCH.show()

+-----------+------------+-------------------+-----------------+------------+----------+--------------+-------------------+
|BRANCH_CODE| BRANCH_NAME|      BRANCH_STREET|      BRANCH_CITY|BRANCH_STATE|BRANCH_ZIP|  BRANCH_PHONE|       LAST_UPDATED|
+-----------+------------+-------------------+-----------------+------------+----------+--------------+-------------------+
|          1|Example Bank|       Bridle Court|        Lakeville|          MN|     55044|(123) 456-5276|2018-04-18 16:51:47|
|          2|Example Bank|  Washington Street|          Huntley|          IL|     60142|(123) 461-8993|2018-04-18 16:51:47|
|          3|Example Bank|      Warren Street|SouthRichmondHill|          NY|     11419|(123) 498-5926|2018-04-18 16:51:47|
|          4|Example Bank|   Cleveland Street|       Middleburg|          FL|     32068|(123) 466-3064|2018-04-18 16:51:47|
|          5|Example Bank|        14th Street|    KingOfPrussia|          PA|     19406|(123) 484-9701|2018-04-18 16:51:47|
|       

### Checking the Data Types

In [10]:
CDW_SAPP_BRANCH.dtypes

[('BRANCH_CODE', 'int'),
 ('BRANCH_NAME', 'string'),
 ('BRANCH_STREET', 'string'),
 ('BRANCH_CITY', 'string'),
 ('BRANCH_STATE', 'string'),
 ('BRANCH_ZIP', 'int'),
 ('BRANCH_PHONE', 'string'),
 ('LAST_UPDATED', 'timestamp')]

### Load data into MYSQL Database called creditcard_capstone

In [11]:
CDW_SAPP_BRANCH.write.format("jdbc") \
  .mode("append") \
  .option("url", "jdbc:mysql://localhost:3306/creditcard_capstone") \
  .option("dbtable", "CDW_SAPP_BRANCH") \
  .option("user", "root") \
  .option("password", "password") \
  .save()